In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Introduction

In this project, we use the Metropolis-Hastings algorithm to decode a message that was encypted by a random or custom generated cypher. The Metropolis-Hastings algorithm is a Monte-Carlo-Markov-Chain algorith, hence the name of this file.

First, we create a list of the frequency of words